In [92]:
import gc
import os
import sys
import math
import random
import warnings
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)
import optuna
from tqdm import tqdm
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from pandarallel import pandarallel
# pandarallel.initialize()
warnings.simplefilter("ignore")
import ctypes as ct
from datetime import timedelta

from sklearn.metrics import mean_absolute_error

import statistics as st
import lightgbm as lgbm
from scipy.stats import norm

In [93]:
sys.path.append('../../')
import src.utils as utils

## Param

In [94]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName', 'birthCity', 'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight', 'playerForTestSetAndFuturePreds']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
salaries_cols = ['teamId', 'salary', 'year']
standings_cols = ['teamId', 'wildCardRank', 'sportGamesBack', 'date']
transactions_cols = ['playerId', 'transaction_flag', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date']

feature_cols1 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info', 'wildCardRank', 'label_awardName'] 

feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info', 'wildCardRank', 'label_awardName'] 

feature_cols3 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info', 'wildCardRank', 'label_awardName'] 

feature_cols4 = ['week_day', 'annual_day', 'month', 'label_playerId', 'label_primaryPositionName', 'label_teamId', 'label_birthCity',
                'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info', 'wildCardRank', 'label_awardName'] 

In [95]:
EXP_NUM = 61
NFOLDS = 5
SEED = 42

In [96]:
utils.set_seed(SEED)

## Dir

In [97]:
DATA_DIR = Path("/home/knikaido/work/MLB-Player-Digital-Engagement-Forecasting/data/")
MAIN_DATA_DIR = DATA_DIR / 'mlb-player-digital-engagement-forecasting'
TRAIN_DIR = MAIN_DATA_DIR / 'train'
OUTPUT_DIR = Path('./output/')

In [98]:
players = pd.read_csv(MAIN_DATA_DIR / 'players.csv')

rosters = pd.read_csv(TRAIN_DIR / 'rosters_train.csv')
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv')
scores = pd.read_csv(TRAIN_DIR / 'playerBoxScores_train.csv')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
seasons = pd.read_csv(MAIN_DATA_DIR / 'seasons.csv')
salaries = pd.read_csv(MAIN_DATA_DIR / 'mlbSalaries.csv')
teams = pd.read_csv(MAIN_DATA_DIR / 'teams.csv')

standings = pd.read_csv(TRAIN_DIR / 'standings_train.csv')
awards = pd.read_csv(TRAIN_DIR / 'awards_train.csv')

playerTwitterFollowers = pd.read_csv(TRAIN_DIR / 'playerTwitterFollowers_train.csv')

# events = pd.read_csv(TRAIN_DIR / 'events_train.csv')
# events = events.groupby(['gameDate']).sum().reset_index()


In [99]:
scores

,playerId,date,home,gamePk,teamId,jerseyNum,positionCode,battingOrder,gamesPlayedBatting,flyOuts,...,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,index
0,112526,20180402,0,529469,140,40.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,91
1,112526,20180408,1,529546,140,40.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,97
2,112526,20180410,1,529565,140,40.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,99
3,112526,20180415,0,529640,140,40.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,104
4,112526,20180421,1,529718,140,40.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182945,685503,20210409,1,634478,140,35.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1194
182946,685503,20210414,0,634496,140,35.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1199
182947,685503,20210419,0,634536,140,35.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1204
182948,685503,20210425,0,634393,140,35.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1210


In [100]:
seasons = seasons.fillna('0000-00-00')
for c_ in seasons.columns[1:]:
    seasons[c_] = seasons[c_].str.replace('-', '').astype(int)

In [101]:
game_dates_range = []
game_dates_range.append(np.array([20170101, 20171231]))
game_dates_range.append(scores[scores['date'] <= 20181231].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range.append(scores[(scores['date'] >= 20190101) & (scores['date'] <= 20191231)].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range.append(scores[(scores['date'] >= 20200101) & (scores['date'] <= 20201231)].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range.append(scores[(scores['date'] >= 20210101)].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range = np.array(game_dates_range)

In [102]:
seasons['gameStartDate'] = game_dates_range[:, 0]
seasons['gameEndDate'] = game_dates_range[:, 1]

In [103]:
seasons

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate,gameStartDate,gameEndDate
0,2017,20170402,20171101,20170222,20170401,20170402,20171001,20170709,20170711,20170714,20171003,20171101,20170101,20171231
1,2018,20180329,20181028,20180221,20180327,20180329,20181001,20180715,20180717,20180719,20181002,20181028,20180329,20181028
2,2019,20190320,20191030,20190221,20190326,20190320,20190929,20190707,20190709,20190711,20191001,20191030,20190320,20191030
3,2020,20200723,20201028,20200221,20200722,20200723,20200927,20200825,0,20200826,20200929,20201028,20200723,20201027
4,2021,20210228,20211031,20210228,20210330,20210401,20211003,20210711,20210713,20210715,20211004,20211031,20210401,20210430


In [104]:
targets_train = targets.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')

In [105]:
targets_train = targets_train[(targets_train['date'] >= 20210401)]

In [106]:
targets_train

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,teamId,status
2444346,2021-04-02,624415,2.810640,3.998462,0.136137,10.012438,1186,20210401,141.0,Active
2444347,2021-04-02,656548,0.000000,0.153787,0.068889,0.062189,1186,20210401,146.0,Reassigned to Minors
2444348,2021-04-02,400284,0.000971,0.010985,0.021323,3.182007,1186,20210401,NaN,NaN
2444349,2021-04-02,606157,0.002736,1.642226,0.021323,0.549337,1186,20210401,113.0,Active
2444350,2021-04-02,665620,0.020473,0.411929,0.549469,0.435323,1186,20210401,147.0,Reassigned to Minors
...,...,...,...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430,NaN,NaN
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430,NaN,NaN
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430,NaN,NaN
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430,NaN,NaN


In [107]:
playerId_list = targets_train['playerId'].unique()

In [108]:
def calc_corr(df):
    # 相関係数行列を作成
    corr_mat = df.corr(method='pearson')

    # 行（列）サイズを取得
    n = corr_mat.shape[0]
    corr_ary = []

    for i in range(n):
        for j in range(i):
            if i == j:
                continue
            corr_ary.append(corr_mat.iloc[i,j])

    return corr_ary

In [109]:
def calc_probs(pid,df,temp):
    to_append=[pid,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    targets=['target1','target2','target3','target4']
    z=1
    for target in targets:
        target_prob = temp[target].tolist()
        mean = np.mean(target_prob)
        std = np.std(target_prob)
        median = st.median(target_prob)
        distribution = norm(mean, std)
        min_weight = min(target_prob)
        max_weight = max(target_prob)
        values = list(np.linspace(min_weight, max_weight))
        probabilities = [distribution.pdf(v) for v in values]
        max_value = max(probabilities)
        max_index = probabilities.index(max_value)
        to_append[z]=mean
        to_append[z+1]=median
        to_append[z+2]=std
        to_append[z+3]=min_weight
        to_append[z+4]=max_weight
        to_append[z+5]=temp[target].skew()
        to_append[z+6]=temp[target].kurt()

        z=z+7
    corr_ = calc_corr(temp[['target1', 'target2', 'target3', 'target4']])
    to_append[z:] = corr_  
    df_length = len(df)
    df.loc[df_length] = to_append
    return df

    

### CREATE DATAFRAME to store probabilities
column_names = ["playerId", "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
                "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
                "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
                "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt",
                'tgt1_2_corr', 'tgt1_3_corr', 'tgt2_3_corr', 'tgt1_4_corr', 'tgt2_4_corr', 'tgt3_4_corr']
player_target_probs = pd.DataFrame(columns = column_names)
    
for pid in tqdm(playerId_list):
    temp = targets_train[targets_train['playerId'] == pid]
    player_target_stats=calc_probs(pid,player_target_probs,temp)

100%|██████████| 2061/2061 [00:45<00:00, 45.72it/s]


In [110]:
teamId_list = targets_train['teamId'].dropna().unique()

In [111]:
team_target_probs = pd.DataFrame(columns = column_names)
    
for pid in tqdm(teamId_list):
    temp = targets_train[targets_train['teamId'] == pid]
    team_target_stats=calc_probs(pid,team_target_probs,temp)

100%|██████████| 30/30 [00:00<00:00, 43.05it/s]


In [112]:
c_ = list(team_target_stats.columns)
c_ = c_[:1] + ["team_" + word for word in c_[1:]]
team_target_stats.columns = c_

In [113]:
feature_cols1 += c_[1:]
feature_cols2 += c_[1:]
feature_cols3 += c_[1:]
feature_cols4 += c_[1:]

In [114]:
data_names=player_target_stats.columns.values.tolist()

In [115]:
players['DOY'] = pd.to_datetime(players['DOB'], format="%Y-%m-%d").dt.year
players['mlbDebutYear'] = pd.to_datetime(players['mlbDebutDate'], format="%Y-%m-%d").dt.year
players['DebutAge'] = players['mlbDebutYear'] - players['DOY']

In [116]:
# creat dataset
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
train = train.merge(standings[standings_cols], on=['teamId', 'date'], how='left')
train = train.merge(team_target_stats, how='left', left_on=["teamId"],right_on=["playerId"], suffixes=('', 'team_'))
train = train.merge(awards, on=['playerId', 'date'], how='left')


In [117]:
# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
birthCityn2num = {c: i for i, c in enumerate(train['birthCity'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
awardName2num = {c: i for i, c in enumerate(train['awardName'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_birthCity'] = train['birthCity'].map(birthCityn2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)
train['label_awardName'] = train['awardName'].map(awardName2num)
train['label_awardName'] = (train['label_awardName'] != 0) * 1

In [118]:
date_ = pd.to_datetime(train['date'], format="%Y%m%d")
train['annual_day'] = (date_ - pd.to_datetime(date_.dt.year, format="%Y")) /  timedelta(days=1)
train['week_day'] = date_.dt.weekday
train['month'] = date_.dt.month

In [119]:
train['gameday'] = ~train['battingOrder'].isna()*1

In [120]:
train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

In [121]:
def count_consecutive_items_n_cols(df, col_name_list, output_col):
    cum_sum_list = [
        (df[col_name] != df[col_name].shift(1)).cumsum().tolist() for col_name in col_name_list
    ]
    df[output_col] = df.groupby(
        ["_".join(map(str, x)) for x in zip(*cum_sum_list)]
    ).cumcount() + 1
    return df

In [122]:
train=count_consecutive_items_n_cols(train,['playerId','gameday'],'daysSinceLastGame')
train.loc[train['gameday']==1,'daysSinceLastGame']=0

In [123]:
train_game = train[train['gameday']==1]
train_last_game = train_game[~train_game.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
train_last_game.columns = ['playerId', 'lastdate']
train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
train_last_game = pd.merge(train_player_unique, train_last_game, on=['playerId'], how='left' )
train_last_game = train_last_game.fillna(20171231)

In [124]:
train_last_game.sort_values('lastdate')

,playerId,lastdate
856,593590,20171231.0
1995,670462,20171231.0
2000,670764,20171231.0
1765,661269,20171231.0
1932,667674,20171231.0
...,...,...
473,542932,20210430.0
1480,641856,20210430.0
1481,641857,20210430.0
483,543037,20210430.0


In [125]:
def extract_season(date_raw, season_start_end):
    idxes = 0
    for raw in season_start_end.iloc():
        idx_ = ((date_raw >= raw.iloc[0]) & (date_raw <= raw.iloc[1])) * 1
        idxes += idx_
    return idxes

In [126]:
on_preseason_idxes = extract_season(train['date'], seasons[['preSeasonStartDate', 'preSeasonEndDate']])
on_season_idxes = extract_season(train['date'], seasons[['regularSeasonStartDate', 'regularSeasonEndDate']]) * 2
on_postseason_idxes = extract_season(train['date'], seasons[['postSeasonStartDate', 'postSeasonEndDate']]) * 3

special_days = seasons['lastDate1stHalf'].to_list() + seasons['allStarDate'].to_list() + seasons['firstDate2ndHalf'].to_list()
special_idxes = 0
for day in special_days:
    special_idxes += (train['date'] == day) * 4

on_total_season_idxes = on_preseason_idxes
on_total_season_idxes[on_season_idxes==2] = 2
on_total_season_idxes[on_postseason_idxes==3] = 3
on_total_season_idxes[special_idxes==4] = 4

train['season_info'] = on_total_season_idxes

In [127]:
## only on season
on_whole_idxes = extract_season(train['date'], seasons[['seasonStartDate', 'seasonEndDate']])
train = train[on_whole_idxes == 1].reset_index(drop=True)

In [128]:
train = train[train['playerForTestSetAndFuturePreds']==True].reset_index(drop=True)

In [129]:
train

,playerId,target1,target2,target3,target4,date,primaryPositionName,birthCity,DOY,mlbDebutYear,...,label_birthCity,label_teamId,label_status,label_awardName,annual_day,week_day,month,gameday,daysSinceLastGame,season_info
0,405395,1.916353,21.512858,0.078997,7.389917,20180329,First Base,Santo Domingo,1980,2001.0,...,52,8,0,0,87.0,3,3,1,0,2
1,405395,2.431219,38.280184,0.080521,10.307364,20180330,First Base,Santo Domingo,1980,2001.0,...,52,8,0,0,88.0,4,3,1,0,2
2,405395,4.890499,18.149785,0.196634,19.190820,20180331,First Base,Santo Domingo,1980,2001.0,...,52,8,0,0,89.0,5,3,1,0,2
3,405395,0.061685,21.539123,0.215855,28.610586,20180401,First Base,Santo Domingo,1980,2001.0,...,52,8,0,0,90.0,6,4,1,0,2
4,405395,0.021304,19.234421,0.217329,20.419970,20180402,First Base,Santo Domingo,1980,2001.0,...,52,8,0,0,91.0,0,4,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711059,685503,0.044617,1.224728,0.009437,0.737463,20210426,Pitcher,Hiroshima,1992,2021.0,...,902,7,0,0,115.0,0,4,0,1,2
711060,685503,0.019123,1.178880,0.013161,0.790301,20210427,Pitcher,Hiroshima,1992,2021.0,...,902,7,0,0,116.0,1,4,0,2,2
711061,685503,0.015799,4.323489,0.002350,0.970273,20210428,Pitcher,Hiroshima,1992,2021.0,...,902,7,0,0,117.0,2,4,0,3,2
711062,685503,0.018770,31.946021,0.305491,5.938273,20210429,Pitcher,Hiroshima,1992,2021.0,...,902,7,0,0,118.0,3,4,0,4,2


In [130]:
# def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
#     oof_pred = np.zeros(len(y_valid), dtype=np.float32)
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(X_train, y_train), (x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
#     oof_pred = model.predict(x_valid)
#     oof_pred = np.clip(oof_pred, 0, 100)
#     score = mean_absolute_error(oof_pred, y_valid)
#     print('mae:', score)
#     return oof_pred, model, score


In [131]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, learning_rates, verbose=100):
    def opt(trial):
        params = {
                'random_state': SEED,
                'objective':'mae',
                'n_estimators': 10000,
                'learning_rate': 0.1,
                'max_depth': trial.suggest_int('max_depth', 1, 20),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
                'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
                'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
                'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 20),
                'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        }

        model_opt = lgbm.LGBMRegressor(**params)

        model_opt.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_valid, y_valid)],  
            early_stopping_rounds=verbose, 
            verbose=verbose)
        oof_pred = model_opt.predict(x_valid)
        oof_pred = np.clip(oof_pred, 0, 100)
        score = mean_absolute_error(oof_pred, y_valid)
        return -score
    return opt

In [132]:
train_X = train
train_y = train[['target1', 'target2', 'target3', 'target4']]
tr_idxs = []
val_idxs = []

In [133]:
# tr_idx = (train['date'].astype(int) < 20200801)
# val_idx = (train['date'].astype(int) >= 20200801) & (train['date'].astype(int) < 20200901)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

# tr_idx = (train['date'].astype(int) < 20200901)
# val_idx = (train['date'].astype(int) >= 20200901) & (train['date'].astype(int) < 20201001)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

# tr_idx = (train['date'].astype(int) < 20201001)
# val_idx = (train['date'].astype(int) >= 20201001) & (train['date'].astype(int) < 20201028)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

# tr_idx = (train['date'].astype(int) < 20210228)
# val_idx = (train['date'].astype(int) >= 20210228) & (train['date'].astype(int) < 20210401)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

tr_idx = (train['date'].astype(int) < 20210401)
val_idx = ~tr_idx
tr_idxs.append(tr_idx)
val_idxs.append(val_idx)

In [134]:
idx = 0

tr_idx = tr_idxs[idx]
val_idx = val_idxs[idx]

x_train = train_X.loc[tr_idx].reset_index(drop=True)
y_train = train_y.loc[tr_idx].reset_index(drop=True)
x_valid = train_X.loc[val_idx].reset_index(drop=True)
y_valid = train_y.loc[val_idx].reset_index(drop=True)

In [135]:
learning_rates = [0.1, 0.1, 0.1, 0.1]

In [136]:
study1 = optuna.create_study(direction='maximize')
study1.optimize(fit_lgbm(x_train[feature_cols1], y_train['target1'], x_valid[feature_cols1], y_valid['target1'], learning_rates[0]), n_trials=100)

print('Number of finished trials:', len(study1.trials))
print('Best trial:', study1.best_trial.params)

[I 2021-07-11 00:16:24,626] A new study created in memory with name: no-name-ca3af0ad-85c6-4476-874d-128c689d8625


[LightGBM] [Warning] feature_fraction is set=0.3329647255981244, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.3329647255981244
[LightGBM] [Warning] bagging_fraction is set=0.6418305083939903, subsample=0.9 will be ignored. Current value: bagging_fraction=0.6418305083939903
[LightGBM] [Warning] bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.829647	valid_1's l1: 1.15414
[200]	training's l1: 0.817527	valid_1's l1: 1.15094
[300]	training's l1: 0.809633	valid_1's l1: 1.15058
Early stopping, best iteration is:
[276]	training's l1: 0.811926	valid_1's l1: 1.14973


[I 2021-07-11 00:16:45,657] Trial 0 finished with value: -1.1492157334286164 and parameters: {'max_depth': 15, 'min_child_weight': 11, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_lambda': 182.3933102363186, 'reg_alpha': 0.005046747359509982, 'feature_fraction': 0.3329647255981244, 'bagging_fraction': 0.6418305083939903, 'bagging_freq': 17, 'num_leaves': 695, 'min_child_samples': 42}. Best is trial 0 with value: -1.1492157334286164.


[LightGBM] [Warning] feature_fraction is set=0.27073685478381937, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.27073685478381937
[LightGBM] [Warning] bagging_fraction is set=0.2060553334311126, subsample=0.9 will be ignored. Current value: bagging_fraction=0.2060553334311126
[LightGBM] [Warning] bagging_freq is set=12, subsample_freq=0 will be ignored. Current value: bagging_freq=12
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.882566	valid_1's l1: 1.20506
[200]	training's l1: 0.875209	valid_1's l1: 1.19119
[300]	training's l1: 0.874981	valid_1's l1: 1.19052
[400]	training's l1: 0.874878	valid_1's l1: 1.19006


[I 2021-07-11 00:16:50,838] Trial 1 finished with value: -1.1899710464366138 and parameters: {'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 0.014339246434502653, 'reg_alpha': 0.017285700102323824, 'feature_fraction': 0.27073685478381937, 'bagging_fraction': 0.2060553334311126, 'bagging_freq': 12, 'num_leaves': 888, 'min_child_samples': 88}. Best is trial 0 with value: -1.1492157334286164.


...

[LightGBM] [Warning] feature_fraction is set=0.40064892642204675, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.40064892642204675
[LightGBM] [Warning] bagging_fraction is set=0.28226648084733297, subsample=0.8 will be ignored. Current value: bagging_fraction=0.28226648084733297
[LightGBM] [Warning] bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.826348	valid_1's l1: 1.1515
[200]	training's l1: 0.812856	valid_1's l1: 1.15097
Early stopping, best iteration is:
[164]	training's l1: 0.816501	valid_1's l1: 1.1496


[I 2021-07-11 00:56:16,945] Trial 98 finished with value: -1.148296210374439 and parameters: {'max_depth': 19, 'min_child_weight': 15, 'subsample': 0.8, 'colsample_bytree': 0.5, 'reg_lambda': 0.0678864909451595, 'reg_alpha': 0.003769299778264613, 'feature_fraction': 0.40064892642204675, 'bagging_fraction': 0.28226648084733297, 'bagging_freq': 15, 'num_leaves': 678, 'min_child_samples': 25}. Best is trial 72 with value: -1.1403118694046963.


[LightGBM] [Warning] feature_fraction is set=0.3371917974307834, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.3371917974307834
[LightGBM] [Warning] bagging_fraction is set=0.9832689481334378, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9832689481334378
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.813995	valid_1's l1: 1.14872
[200]	training's l1: 0.792617	valid_1's l1: 1.14628
Early stopping, best iteration is:
[184]	training's l1: 0.795862	valid_1's l1: 1.14555


[I 2021-07-11 00:56:40,463] Trial 99 finished with value: -1.1452475642789732 and parameters: {'max_depth': 18, 'min_child_weight': 14, 'subsample': 0.8, 'colsample_bytree': 0.5, 'reg_lambda': 0.12175273270813615, 'reg_alpha': 0.005408777805975193, 'feature_fraction': 0.3371917974307834, 'bagging_fraction': 0.9832689481334378, 'bagging_freq': 20, 'num_leaves': 809, 'min_child_samples': 46}. Best is trial 72 with value: -1.1403118694046963.


Number of finished trials: 100
Best trial: {'max_depth': 18, 'min_child_weight': 8, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 0.07770060902509043, 'reg_alpha': 0.006917721077349456, 'feature_fraction': 0.5448452017199268, 'bagging_fraction': 0.9712581844886307, 'bagging_freq': 17, 'num_leaves': 757, 'min_child_samples': 38}


In [137]:
study2 = optuna.create_study(direction='maximize')
study2.optimize(fit_lgbm(x_train[feature_cols2], y_train['target2'], x_valid[feature_cols2], y_valid['target2'], learning_rates[1]), n_trials=100)

print('Number of finished trials:', len(study2.trials))
print('Best trial:', study2.best_trial.params)

[I 2021-07-11 00:56:40,483] A new study created in memory with name: no-name-28024c76-0f18-48e7-b41e-372641e05a8e


[LightGBM] [Warning] feature_fraction is set=0.5356563179133227, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5356563179133227
[LightGBM] [Warning] bagging_fraction is set=0.9541833930969124, subsample=0.5 will be ignored. Current value: bagging_fraction=0.9541833930969124
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.74735	valid_1's l1: 2.40563


[I 2021-07-11 00:56:50,586] Trial 0 finished with value: -2.317331430817083 and parameters: {'max_depth': 14, 'min_child_weight': 13, 'subsample': 0.5, 'colsample_bytree': 0.8, 'reg_lambda': 1.110206358354379, 'reg_alpha': 0.0272902177815835, 'feature_fraction': 0.5356563179133227, 'bagging_fraction': 0.9541833930969124, 'bagging_freq': 14, 'num_leaves': 519, 'min_child_samples': 83}. Best is trial 0 with value: -2.317331430817083.


Early stopping, best iteration is:
[19]	training's l1: 1.92627	valid_1's l1: 2.31733
[LightGBM] [Warning] feature_fraction is set=0.3249015086298732, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.3249015086298732
[LightGBM] [Warning] bagging_fraction is set=0.4174166983351393, subsample=0.9 will be ignored. Current value: bagging_fraction=0.4174166983351393
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 2.07635	valid_1's l1: 2.33576
[200]	training's l1: 2.04131	valid_1's l1: 2.3158
[300]	training's l1: 2.0188	valid_1's l1: 2.31239


[I 2021-07-11 00:56:57,156] Trial 1 finished with value: -2.3101742747023546 and parameters: {'max_depth': 2, 'min_child_weight': 17, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_lambda': 0.20887909696160611, 'reg_alpha': 35.775780293180766, 'feature_fraction': 0.3249015086298732, 'bagging_fraction': 0.4174166983351393, 'bagging_freq': 6, 'num_leaves': 414, 'min_child_samples': 15}. Best is trial 1 with value: -2.3101742747023546.


...

Early stopping, best iteration is:
[18]	training's l1: 1.96981	valid_1's l1: 2.29276
[LightGBM] [Warning] feature_fraction is set=0.5457685380823484, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.5457685380823484
[LightGBM] [Warning] bagging_fraction is set=0.36808276239955784, subsample=0.7 will be ignored. Current value: bagging_fraction=0.36808276239955784
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.78398	valid_1's l1: 2.3765


[I 2021-07-11 01:05:17,749] Trial 98 finished with value: -2.305713896044115 and parameters: {'max_depth': 11, 'min_child_weight': 20, 'subsample': 0.7, 'colsample_bytree': 0.6, 'reg_lambda': 0.5523472554968012, 'reg_alpha': 0.017210792488903604, 'feature_fraction': 0.5457685380823484, 'bagging_fraction': 0.36808276239955784, 'bagging_freq': 4, 'num_leaves': 555, 'min_child_samples': 71}. Best is trial 78 with value: -2.285454771702014.


Early stopping, best iteration is:
[22]	training's l1: 1.89869	valid_1's l1: 2.30571
[LightGBM] [Warning] feature_fraction is set=0.6308262235730818, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.6308262235730818
[LightGBM] [Warning] bagging_fraction is set=0.24156917327226257, subsample=0.6 will be ignored. Current value: bagging_fraction=0.24156917327226257
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.84164	valid_1's l1: 2.33458


[I 2021-07-11 01:05:21,609] Trial 99 finished with value: -2.294644245824871 and parameters: {'max_depth': 8, 'min_child_weight': 19, 'subsample': 0.6, 'colsample_bytree': 0.5, 'reg_lambda': 0.3235622638700643, 'reg_alpha': 0.05482553143363373, 'feature_fraction': 0.6308262235730818, 'bagging_fraction': 0.24156917327226257, 'bagging_freq': 5, 'num_leaves': 469, 'min_child_samples': 78}. Best is trial 78 with value: -2.285454771702014.


Early stopping, best iteration is:
[21]	training's l1: 1.94863	valid_1's l1: 2.29464
Number of finished trials: 100
Best trial: {'max_depth': 9, 'min_child_weight': 19, 'subsample': 0.7, 'colsample_bytree': 0.5, 'reg_lambda': 0.842789768469053, 'reg_alpha': 0.002083027512061945, 'feature_fraction': 0.6685585785820495, 'bagging_fraction': 0.24812681263609027, 'bagging_freq': 4, 'num_leaves': 300, 'min_child_samples': 88}


In [138]:
study3 = optuna.create_study(direction='maximize')
study3.optimize(fit_lgbm(x_train[feature_cols3], y_train['target3'], x_valid[feature_cols3], y_valid['target3'], learning_rates[2]), n_trials=100)

print('Number of finished trials:', len(study3.trials))
print('Best trial:', study3.best_trial.params)

[I 2021-07-11 01:05:21,629] A new study created in memory with name: no-name-3ed46c1a-523f-4604-94d2-c2796ad1f2e5


[LightGBM] [Warning] feature_fraction is set=0.8010058286178012, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.8010058286178012
[LightGBM] [Warning] bagging_fraction is set=0.2655683981657584, subsample=0.8 will be ignored. Current value: bagging_fraction=0.2655683981657584
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.02802	valid_1's l1: 0.940229
[200]	training's l1: 1.02636	valid_1's l1: 0.938505
[300]	training's l1: 1.02544	valid_1's l1: 0.938052
[400]	training's l1: 1.02518	valid_1's l1: 0.937863


[I 2021-07-11 01:05:30,997] Trial 0 finished with value: -0.9377932472955953 and parameters: {'max_depth': 13, 'min_child_weight': 9, 'subsample': 0.8, 'colsample_bytree': 0.6, 'reg_lambda': 2.8857285007679416, 'reg_alpha': 0.695191296239525, 'feature_fraction': 0.8010058286178012, 'bagging_fraction': 0.2655683981657584, 'bagging_freq': 4, 'num_leaves': 83, 'min_child_samples': 54}. Best is trial 0 with value: -0.9377932472955953.


Early stopping, best iteration is:
[326]	training's l1: 1.02524	valid_1's l1: 0.937795
[LightGBM] [Warning] feature_fraction is set=0.8453783826759187, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.8453783826759187
[LightGBM] [Warning] bagging_fraction is set=0.47401121333500584, subsample=0.5 will be ignored. Current value: bagging_fraction=0.47401121333500584
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.0055	valid_1's l1: 0.944396


[I 2021-07-11 01:05:38,059] Trial 1 finished with value: -0.9410826884517539 and parameters: {'max_depth': 17, 'min_child_weight': 11, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_lambda': 0.03706607555499961, 'reg_alpha': 61.77686021233244, 'feature_fraction': 0.8453783826759187, 'bagging_fraction': 0.47401121333500584, 'bagging_freq': 6, 'num_leaves': 826, 'min_child_samples': 54}. Best is trial 0 with value: -0.9377932472955953.


...

[500]	training's l1: 1.00843	valid_1's l1: 0.924632
Early stopping, best iteration is:
[400]	training's l1: 1.00954	valid_1's l1: 0.923813
[LightGBM] [Warning] feature_fraction is set=0.9465155217700669, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.9465155217700669
[LightGBM] [Warning] bagging_fraction is set=0.9622139997224137, subsample=0.7 will be ignored. Current value: bagging_fraction=0.9622139997224137
[LightGBM] [Warning] bagging_freq is set=18, subsample_freq=0 will be ignored. Current value: bagging_freq=18
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.02142	valid_1's l1: 0.932301
[200]	training's l1: 1.01659	valid_1's l1: 0.929125
[300]	training's l1: 1.0136	valid_1's l1: 0.92753
[400]	training's l1: 1.01185	valid_1's l1: 0.92684
[500]	training's l1: 1.01094	valid_1's l1: 0.926589


[I 2021-07-11 01:22:18,788] Trial 98 finished with value: -0.9264729312774322 and parameters: {'max_depth': 6, 'min_child_weight': 8, 'subsample': 0.7, 'colsample_bytree': 0.9, 'reg_lambda': 0.8120633720071807, 'reg_alpha': 0.13010119672490533, 'feature_fraction': 0.9465155217700669, 'bagging_fraction': 0.9622139997224137, 'bagging_freq': 18, 'num_leaves': 812, 'min_child_samples': 28}. Best is trial 97 with value: -0.9238043197782138.


Early stopping, best iteration is:
[450]	training's l1: 1.01105	valid_1's l1: 0.926497
[LightGBM] [Warning] feature_fraction is set=0.9464359149049805, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.9464359149049805
[LightGBM] [Warning] bagging_fraction is set=0.9970806949934554, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9970806949934554
[LightGBM] [Warning] bagging_freq is set=18, subsample_freq=0 will be ignored. Current value: bagging_freq=18
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.02958	valid_1's l1: 0.937428
[200]	training's l1: 1.02556	valid_1's l1: 0.934815
[300]	training's l1: 1.02177	valid_1's l1: 0.931887
[400]	training's l1: 1.0203	valid_1's l1: 0.930764
[500]	training's l1: 1.01897	valid_1's l1: 0.929197
[600]	training's l1: 1.01848	valid_1's l1: 0.929338


[I 2021-07-11 01:22:36,185] Trial 99 finished with value: -0.9291190543822962 and parameters: {'max_depth': 4, 'min_child_weight': 8, 'subsample': 0.8, 'colsample_bytree': 0.9, 'reg_lambda': 1.0248933889409264, 'reg_alpha': 0.12430216601743362, 'feature_fraction': 0.9464359149049805, 'bagging_fraction': 0.9970806949934554, 'bagging_freq': 18, 'num_leaves': 804, 'min_child_samples': 31}. Best is trial 97 with value: -0.9238043197782138.


Early stopping, best iteration is:
[520]	training's l1: 1.01895	valid_1's l1: 0.929136
Number of finished trials: 100
Best trial: {'max_depth': 6, 'min_child_weight': 8, 'subsample': 0.6, 'colsample_bytree': 0.9, 'reg_lambda': 0.20136897056918007, 'reg_alpha': 0.12085385697554701, 'feature_fraction': 0.9511894033022753, 'bagging_fraction': 0.9078085686770417, 'bagging_freq': 18, 'num_leaves': 808, 'min_child_samples': 35}


In [139]:
study4 = optuna.create_study(direction='maximize')
study4.optimize(fit_lgbm(x_train[feature_cols4], y_train['target4'], x_valid[feature_cols4], y_valid['target4'], learning_rates[3]), n_trials=100)

print('Number of finished trials:', len(study4.trials))
print('Best trial:', study4.best_trial.params)

[I 2021-07-11 01:22:36,205] A new study created in memory with name: no-name-5aedb9fb-e594-4827-8b92-7721158c553d


[LightGBM] [Warning] feature_fraction is set=0.5972393864123822, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.5972393864123822
[LightGBM] [Warning] bagging_fraction is set=0.6037530103137048, subsample=0.7 will be ignored. Current value: bagging_fraction=0.6037530103137048
[LightGBM] [Warning] bagging_freq is set=19, subsample_freq=0 will be ignored. Current value: bagging_freq=19
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.06313	valid_1's l1: 1.63855
[200]	training's l1: 1.04959	valid_1's l1: 1.62429
[300]	training's l1: 1.04301	valid_1's l1: 1.61709
[400]	training's l1: 1.03638	valid_1's l1: 1.61112
[500]	training's l1: 1.02335	valid_1's l1: 1.60002
[600]	training's l1: 1.01597	valid_1's l1: 1.60012


[I 2021-07-11 01:22:53,084] Trial 0 finished with value: -1.5986318225863427 and parameters: {'max_depth': 3, 'min_child_weight': 4, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_lambda': 24.29711922213026, 'reg_alpha': 0.4913963330753671, 'feature_fraction': 0.5972393864123822, 'bagging_fraction': 0.6037530103137048, 'bagging_freq': 19, 'num_leaves': 874, 'min_child_samples': 34}. Best is trial 0 with value: -1.5986318225863427.


Early stopping, best iteration is:
[531]	training's l1: 1.02146	valid_1's l1: 1.59876
[LightGBM] [Warning] feature_fraction is set=0.2817480113795829, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.2817480113795829
[LightGBM] [Warning] bagging_fraction is set=0.5926863812069976, subsample=0.7 will be ignored. Current value: bagging_fraction=0.5926863812069976
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.877553	valid_1's l1: 1.58247
[200]	training's l1: 0.847506	valid_1's l1: 1.57804


[I 2021-07-11 01:23:06,204] Trial 1 finished with value: -1.5750764307892247 and parameters: {'max_depth': 14, 'min_child_weight': 10, 'subsample': 0.7, 'colsample_bytree': 0.5, 'reg_lambda': 0.0024873494534892885, 'reg_alpha': 3.379708367327537, 'feature_fraction': 0.2817480113795829, 'bagging_fraction': 0.5926863812069976, 'bagging_freq': 8, 'num_leaves': 406, 'min_child_samples': 44}. Best is trial 1 with value: -1.5750764307892247.


...

Early stopping, best iteration is:
[134]	training's l1: 0.968975	valid_1's l1: 1.58338
[LightGBM] [Warning] feature_fraction is set=0.6115281605839483, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6115281605839483
[LightGBM] [Warning] bagging_fraction is set=0.22420959735637974, subsample=0.9 will be ignored. Current value: bagging_fraction=0.22420959735637974
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.890657	valid_1's l1: 1.57908


[I 2021-07-11 01:42:49,648] Trial 98 finished with value: -1.5728961355178217 and parameters: {'max_depth': 17, 'min_child_weight': 14, 'subsample': 0.9, 'colsample_bytree': 0.6, 'reg_lambda': 2.2922542997585174, 'reg_alpha': 44.00071164284988, 'feature_fraction': 0.6115281605839483, 'bagging_fraction': 0.22420959735637974, 'bagging_freq': 9, 'num_leaves': 503, 'min_child_samples': 73}. Best is trial 87 with value: -1.5669597137505078.


Early stopping, best iteration is:
[63]	training's l1: 0.909107	valid_1's l1: 1.57305
[LightGBM] [Warning] feature_fraction is set=0.7068917107933401, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.7068917107933401
[LightGBM] [Warning] bagging_fraction is set=0.386499754914871, subsample=0.9 will be ignored. Current value: bagging_fraction=0.386499754914871
[LightGBM] [Warning] bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.844114	valid_1's l1: 1.58551


[I 2021-07-11 01:42:57,865] Trial 99 finished with value: -1.577905800914352 and parameters: {'max_depth': 15, 'min_child_weight': 16, 'subsample': 0.9, 'colsample_bytree': 0.6, 'reg_lambda': 1.0553747302687568, 'reg_alpha': 11.92615170422777, 'feature_fraction': 0.7068917107933401, 'bagging_fraction': 0.386499754914871, 'bagging_freq': 13, 'num_leaves': 684, 'min_child_samples': 85}. Best is trial 87 with value: -1.5669597137505078.


Early stopping, best iteration is:
[63]	training's l1: 0.863332	valid_1's l1: 1.57801
Number of finished trials: 100
Best trial: {'max_depth': 16, 'min_child_weight': 19, 'subsample': 0.8, 'colsample_bytree': 0.6, 'reg_lambda': 0.9189086491671924, 'reg_alpha': 85.91747956919798, 'feature_fraction': 0.653862546945534, 'bagging_fraction': 0.2539878828275932, 'bagging_freq': 12, 'num_leaves': 300, 'min_child_samples': 92}


In [140]:
study1.best_trial.params

{'max_depth': 18,
 'min_child_weight': 8,
 'subsample': 0.9,
 'colsample_bytree': 0.5,
 'reg_lambda': 0.07770060902509043,
 'reg_alpha': 0.006917721077349456,
 'feature_fraction': 0.5448452017199268,
 'bagging_fraction': 0.9712581844886307,
 'bagging_freq': 17,
 'num_leaves': 757,
 'min_child_samples': 38}

In [141]:
study2.best_trial.params

{'max_depth': 9,
 'min_child_weight': 19,
 'subsample': 0.7,
 'colsample_bytree': 0.5,
 'reg_lambda': 0.842789768469053,
 'reg_alpha': 0.002083027512061945,
 'feature_fraction': 0.6685585785820495,
 'bagging_fraction': 0.24812681263609027,
 'bagging_freq': 4,
 'num_leaves': 300,
 'min_child_samples': 88}

In [142]:
study3.best_trial.params

{'max_depth': 6,
 'min_child_weight': 8,
 'subsample': 0.6,
 'colsample_bytree': 0.9,
 'reg_lambda': 0.20136897056918007,
 'reg_alpha': 0.12085385697554701,
 'feature_fraction': 0.9511894033022753,
 'bagging_fraction': 0.9078085686770417,
 'bagging_freq': 18,
 'num_leaves': 808,
 'min_child_samples': 35}

In [143]:
study4.best_trial.params

{'max_depth': 16,
 'min_child_weight': 19,
 'subsample': 0.8,
 'colsample_bytree': 0.6,
 'reg_lambda': 0.9189086491671924,
 'reg_alpha': 85.91747956919798,
 'feature_fraction': 0.653862546945534,
 'bagging_fraction': 0.2539878828275932,
 'bagging_freq': 12,
 'num_leaves': 300,
 'min_child_samples': 92}